# OpenRXN Example: Membrane slab

### This notebook demostrates how a complicated system can be set up easily with the OpenRXN package

We are interested in setting up a 3D system with a membrane slab at the bottom (both lower and upper leaflets), and a bulk region on top.  There will be three Species in our model (drug, receptor and drug-receptor complex), with only the drug allowed to move around in the bulk.  The receptor and drug-receptor complex are assumed to be locked in the membrane region.

**Our goal here is to build a model that describes both:**   
1) The diffusion of the drug both in the membrane and the bulk   
2) The binding of the drug to the receptor   

First we import the necessary things from the OpenRXN package:

In [1]:
from openrxn.reactions import Reaction, Species
from openrxn.compartments.arrays import CompartmentArray3D
from openrxn.connections import IsotropicConnection, AnisotropicConnection, FicksConnection
from openrxn.model import Model

import pint
unit = pint.UnitRegistry()

import numpy as np

Then we define the Species objects to use in our model.

In [2]:
drug = Species('drug')
receptor = Species('receptor')
dr_complex  = Species('complex')

In OpenRXN, we can define Reaction objects as follows:

In [ ]:
Reaction?

In [3]:
kon = 1e6/(unit.molar*unit.sec)
koff = 0.1/unit.sec

binding = Reaction('binding',[drug,receptor],[dr_complex],[1,1],[1],kf=kon,kr=koff)

In [4]:
binding.display()

'drug + receptor <--->complex  // kr = 0.1 / second // kf = 1000000.0 / molar / second'

The biggest utility of OpenRXN lies in the ability to create compartments (and arrays of compartments) within which these reactions can occur.  Species can also diffuse between compartments with specified rates.

In [5]:
conn_in_slab = FicksConnection({'drug' : 1e-8*unit.cm**2/unit.sec})

x_pos = np.linspace(-50,50,101)*unit.nanometer
y_pos = np.linspace(-50,50,101)*unit.nanometer
z_pos1 = np.array([-1,0])*unit.nanometer
lower_slab = CompartmentArray3D('lower_slab',x_pos,y_pos,z_pos1,conn_in_slab,periodic=[True,True,False])

This created a 3D array of compartments with positions as specified by x_pos, y_pos and z_pos1.  It is periodic in the x and y dimensions, and the compartments are automatically connected:

In [9]:
lower_slab.compartments[(0,0,0)].connections

{'lower_slab-1_0_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b08b4588>,
  <openrxn.connections.FicksConnection at 0x7ff9c04e4160>),
 'lower_slab-0_1_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b0811278>,
  <openrxn.connections.FicksConnection at 0x7ff9c04e4160>),
 'lower_slab-99_0_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9c0f17320>,
  <openrxn.connections.FicksConnection at 0x7ff9c04e4160>),
 'lower_slab-0_99_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b08b42b0>,
  <openrxn.connections.FicksConnection at 0x7ff9c04e4160>)}

In [10]:
lower_slab.compartments[(0,0,0)].pos

[(-50.0 <Unit('nanometer')>, -49.0 <Unit('nanometer')>),
 (-50.0 <Unit('nanometer')>, -49.0 <Unit('nanometer')>),
 (-1 <Unit('nanometer')>, 0 <Unit('nanometer')>)]

Let's create another 3D array for the upper slab:

In [12]:
z_pos2 = np.array([0,1])*unit.nanometer
upper_slab = CompartmentArray3D('upper_slab',x_pos,y_pos,z_pos2,conn_in_slab,periodic=[True,True,False])

And now we can define another connection type for between the slabs: 

In [13]:
conn_between_slab = IsotropicConnection({'drug' : 1e-5/unit.sec})

Then use the join3D method to make connections between our 3D compartment arrays:

In [14]:
lower_slab.join3D(upper_slab,conn_between_slab,append_side='z+')

Now let's define a bulk compartment array, and connect it to the upper slab:

In [15]:
conn_bulk_to_bulk = FicksConnection({'drug' : 1e-5*unit.cm**2/unit.sec})
z_pos3 = np.linspace(1,25,23)*unit.nanometer

bulk = CompartmentArray3D('bulk',x_pos,y_pos,z_pos3,conn_bulk_to_bulk,periodic=[True,True,False])

conn_slab_to_bulk = AnisotropicConnection({'drug' : (1e-5/unit.sec, 1e-1/unit.sec)})
bulk.join3D(upper_slab,conn_slab_to_bulk,append_side='z-')

Now we can put all this together in a "Model" object:

In [16]:
my_model = Model([lower_slab,upper_slab,bulk])

In [18]:
my_model.arrays

{'lower_slab': <openrxn.compartments.arrays.CompartmentArray3D at 0x7ff9b082dc50>,
 'upper_slab': <openrxn.compartments.arrays.CompartmentArray3D at 0x7ff9d19e5710>,
 'bulk': <openrxn.compartments.arrays.CompartmentArray3D at 0x7ff9a2491dd8>}

In order to visualize the connections between the compartments, and to turn this model into a "system" that can be integrated forward in time, we turn it into a "FlatModel" using the flatten() method:

In [19]:
my_flat_model = my_model.flatten()

In [20]:
len(my_flat_model.compartments)

240000

In [22]:
b000 = my_flat_model.compartments['bulk-0_0_0']

In [23]:
b000.connections

{'bulk-1_0_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b1fc5f98>,
  <openrxn.connections.IsotropicConnection at 0x7ff93347ec88>),
 'bulk-0_1_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b1ec1f28>,
  <openrxn.connections.IsotropicConnection at 0x7ff93347ea58>),
 'bulk-0_0_1': (<openrxn.compartments.compartment.Compartment at 0x7ff9b1eb54e0>,
  <openrxn.connections.IsotropicConnection at 0x7ff93347eb70>),
 'bulk-99_0_0': (<openrxn.compartments.compartment.Compartment at 0x7ff96f42bcc0>,
  <openrxn.connections.IsotropicConnection at 0x7ff93347ed30>),
 'bulk-0_99_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9b1fb72e8>,
  <openrxn.connections.IsotropicConnection at 0x7ff93347edd8>),
 'upper_slab-0_0_0': (<openrxn.compartments.compartment.Compartment at 0x7ff9c151fc18>,
  <openrxn.connections.AnisotropicConnection at 0x7ff96f42ba90>)}